In [1]:
import tensorflow as tf

import numpy as np
import os
import time

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [72]:
%cd gdrive/My\ Drive/NLP/lesson-9

/content/gdrive/My Drive/NLP/lesson-9


In [73]:
!ls *.txt

chekhov.txt  evgenyi_onegin.txt  vishnevyy-sad.txt


In [74]:
path_to_file = 'vishnevyy-sad.txt'

In [75]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 89357 characters


In [124]:
text = text + text + text

In [125]:
print(text[:250])

     Дуняша. А собаки всю ночь не спали, чуют, что хозяева едут.
     Лопахин. Что ты, Дуняша, такая...
     Дуняша. Руки трясутся. Я в обморок упаду.
     Лопахин. Очень уж ты нежная, Дуняша. И одеваешься, как барышня, и прическа тоже. Так нельзя


In [126]:
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

106 unique characters


In [127]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)

In [128]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

In [129]:
def text_from_ids(ids):
    return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [130]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(268071,), dtype=int64, numpy=array([3, 3, 3, ..., 3, 2, 1])>

In [131]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [132]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

 
 
 
 
 
Д
у
н
я
ш


In [133]:
seq_length = 100

In [134]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
    print(chars_from_ids(seq))

tf.Tensor(
[b' ' b' ' b' ' b' ' b' ' b'\xd0\x94' b'\xd1\x83' b'\xd0\xbd' b'\xd1\x8f'
 b'\xd1\x88' b'\xd0\xb0' b'.' b' ' b'\xd0\x90' b' ' b'\xd1\x81'
 b'\xd0\xbe' b'\xd0\xb1' b'\xd0\xb0' b'\xd0\xba' b'\xd0\xb8' b' '
 b'\xd0\xb2' b'\xd1\x81' b'\xd1\x8e' b' ' b'\xd0\xbd' b'\xd0\xbe'
 b'\xd1\x87' b'\xd1\x8c' b' ' b'\xd0\xbd' b'\xd0\xb5' b' ' b'\xd1\x81'
 b'\xd0\xbf' b'\xd0\xb0' b'\xd0\xbb' b'\xd0\xb8' b',' b' ' b'\xd1\x87'
 b'\xd1\x83' b'\xd1\x8e' b'\xd1\x82' b',' b' ' b'\xd1\x87' b'\xd1\x82'
 b'\xd0\xbe' b' ' b'\xd1\x85' b'\xd0\xbe' b'\xd0\xb7' b'\xd1\x8f'
 b'\xd0\xb5' b'\xd0\xb2' b'\xd0\xb0' b' ' b'\xd0\xb5' b'\xd0\xb4'
 b'\xd1\x83' b'\xd1\x82' b'.' b'\r' b'\n' b' ' b' ' b' ' b' ' b' '
 b'\xd0\x9b' b'\xd0\xbe' b'\xd0\xbf' b'\xd0\xb0' b'\xd1\x85' b'\xd0\xb8'
 b'\xd0\xbd' b'.' b' ' b'\xd0\xa7' b'\xd1\x82' b'\xd0\xbe' b' '
 b'\xd1\x82' b'\xd1\x8b' b',' b' ' b'\xd0\x94' b'\xd1\x83' b'\xd0\xbd'
 b'\xd1\x8f' b'\xd1\x88' b'\xd0\xb0' b',' b' ' b'\xd1\x82' b'\xd0\xb0'
 b'\xd0\xba' b'\xd0\xb0' b'\

In [135]:
for seq in sequences.take(5):
    print(text_from_ids(seq).numpy())

b'     \xd0\x94\xd1\x83\xd0\xbd\xd1\x8f\xd1\x88\xd0\xb0. \xd0\x90 \xd1\x81\xd0\xbe\xd0\xb1\xd0\xb0\xd0\xba\xd0\xb8 \xd0\xb2\xd1\x81\xd1\x8e \xd0\xbd\xd0\xbe\xd1\x87\xd1\x8c \xd0\xbd\xd0\xb5 \xd1\x81\xd0\xbf\xd0\xb0\xd0\xbb\xd0\xb8, \xd1\x87\xd1\x83\xd1\x8e\xd1\x82, \xd1\x87\xd1\x82\xd0\xbe \xd1\x85\xd0\xbe\xd0\xb7\xd1\x8f\xd0\xb5\xd0\xb2\xd0\xb0 \xd0\xb5\xd0\xb4\xd1\x83\xd1\x82.\r\n     \xd0\x9b\xd0\xbe\xd0\xbf\xd0\xb0\xd1\x85\xd0\xb8\xd0\xbd. \xd0\xa7\xd1\x82\xd0\xbe \xd1\x82\xd1\x8b, \xd0\x94\xd1\x83\xd0\xbd\xd1\x8f\xd1\x88\xd0\xb0, \xd1\x82\xd0\xb0\xd0\xba\xd0\xb0\xd1\x8f'
b'...\r\n     \xd0\x94\xd1\x83\xd0\xbd\xd1\x8f\xd1\x88\xd0\xb0. \xd0\xa0\xd1\x83\xd0\xba\xd0\xb8 \xd1\x82\xd1\x80\xd1\x8f\xd1\x81\xd1\x83\xd1\x82\xd1\x81\xd1\x8f. \xd0\xaf \xd0\xb2 \xd0\xbe\xd0\xb1\xd0\xbc\xd0\xbe\xd1\x80\xd0\xbe\xd0\xba \xd1\x83\xd0\xbf\xd0\xb0\xd0\xb4\xd1\x83.\r\n     \xd0\x9b\xd0\xbe\xd0\xbf\xd0\xb0\xd1\x85\xd0\xb8\xd0\xbd. \xd0\x9e\xd1\x87\xd0\xb5\xd0\xbd\xd1\x8c \xd1\x83\xd0\xb6 \xd1\x82\xd1\

In [136]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [137]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [138]:
dataset = sequences.map(split_input_target)

In [139]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'     \xd0\x94\xd1\x83\xd0\xbd\xd1\x8f\xd1\x88\xd0\xb0. \xd0\x90 \xd1\x81\xd0\xbe\xd0\xb1\xd0\xb0\xd0\xba\xd0\xb8 \xd0\xb2\xd1\x81\xd1\x8e \xd0\xbd\xd0\xbe\xd1\x87\xd1\x8c \xd0\xbd\xd0\xb5 \xd1\x81\xd0\xbf\xd0\xb0\xd0\xbb\xd0\xb8, \xd1\x87\xd1\x83\xd1\x8e\xd1\x82, \xd1\x87\xd1\x82\xd0\xbe \xd1\x85\xd0\xbe\xd0\xb7\xd1\x8f\xd0\xb5\xd0\xb2\xd0\xb0 \xd0\xb5\xd0\xb4\xd1\x83\xd1\x82.\r\n     \xd0\x9b\xd0\xbe\xd0\xbf\xd0\xb0\xd1\x85\xd0\xb8\xd0\xbd. \xd0\xa7\xd1\x82\xd0\xbe \xd1\x82\xd1\x8b, \xd0\x94\xd1\x83\xd0\xbd\xd1\x8f\xd1\x88\xd0\xb0, \xd1\x82\xd0\xb0\xd0\xba\xd0\xb0'
Target: b'    \xd0\x94\xd1\x83\xd0\xbd\xd1\x8f\xd1\x88\xd0\xb0. \xd0\x90 \xd1\x81\xd0\xbe\xd0\xb1\xd0\xb0\xd0\xba\xd0\xb8 \xd0\xb2\xd1\x81\xd1\x8e \xd0\xbd\xd0\xbe\xd1\x87\xd1\x8c \xd0\xbd\xd0\xb5 \xd1\x81\xd0\xbf\xd0\xb0\xd0\xbb\xd0\xb8, \xd1\x87\xd1\x83\xd1\x8e\xd1\x82, \xd1\x87\xd1\x82\xd0\xbe \xd1\x85\xd0\xbe\xd0\xb7\xd1\x8f\xd0\xb5\xd0\xb2\xd0\xb0 \xd0\xb5\xd0\xb4\xd1\x83\xd1\x82.\r\n     \xd0\x9b\xd0\xbe\xd0

In [140]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [141]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

### train and target

In [204]:
def build_model(vocab_size, embedding_dim, rnn_units):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim),
                                 
        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=False,
                            recurrent_initializer='glorot_uniform'),

        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=False,
                            recurrent_initializer='glorot_uniform'),
      
                                   
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [205]:
model = build_model(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [206]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 107) # (batch_size, sequence_length, vocab_size)


In [207]:
model.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_10 (Embedding)    (None, None, 256)         27392     
                                                                 
 gru_11 (GRU)                (None, None, 1024)        3938304   
                                                                 
 lstm_4 (LSTM)               (None, None, 1024)        8392704   
                                                                 
 dense_9 (Dense)             (None, None, 107)         109675    
                                                                 
Total params: 12,468,075
Trainable params: 12,468,075
Non-trainable params: 0
_________________________________________________________________


In [208]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

## Train the model

In [209]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 107)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.6732244


In [210]:
model.compile(optimizer='adam', loss=loss)

### Configure checkpoints

In [211]:
!rm -rf ./training_checkpoints

In [212]:
!ls ./training_checkpoints

ls: cannot access './training_checkpoints': No such file or directory


In [213]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_freq=88*5,
    save_weights_only=True)

### Execute the training

In [214]:
EPOCHS = 50

In [215]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/50
41/41 [==============================] - 10s 162ms/step - loss: 3.5195
Epoch 2/50
41/41 [==============================] - 7s 167ms/step - loss: 3.0363
Epoch 3/50
41/41 [==============================] - 7s 170ms/step - loss: 2.5271
Epoch 4/50
41/41 [==============================] - 7s 164ms/step - loss: 2.2988
Epoch 5/50
41/41 [==============================] - 7s 164ms/step - loss: 2.1395
Epoch 6/50
41/41 [==============================] - 7s 162ms/step - loss: 1.9976
Epoch 7/50
41/41 [==============================] - 7s 157ms/step - loss: 1.8671
Epoch 8/50
41/41 [==============================] - 7s 157ms/step - loss: 1.7317
Epoch 9/50
41/41 [==============================] - 7s 158ms/step - loss: 1.5950
Epoch 10/50
41/41 [==============================] - 7s 161ms/step - loss: 1.4537
Epoch 11/50
41/41 [==============================] - 8s 179ms/step - loss: 1.3102
Epoch 12/50
41/41 [==============================] - 7s 166ms/step - loss: 1.1614
Epoch 13/50
41/41 [=====

## Generate text

In [216]:
class OneStep(tf.keras.Model):
    def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
        super().__init__()
        self.temperature = temperature
        self.model = model
        self.chars_from_ids = chars_from_ids
        self.ids_from_chars = ids_from_chars

        # Create a mask to prevent "[UNK]" from being generated.
        skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
        sparse_mask = tf.SparseTensor(
            # Put a -inf at each bad index.
            values=[-float('inf')]*len(skip_ids),
            indices=skip_ids,
            # Match the shape to the vocabulary
            dense_shape=[len(ids_from_chars.get_vocabulary())])
        self.prediction_mask = tf.sparse.to_dense(sparse_mask)

    @tf.function
    def generate_one_step(self, inputs):
        # Convert strings to token IDs.
        input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
        input_ids = self.ids_from_chars(input_chars).to_tensor()

        # Run the model.
        # predicted_logits.shape is [batch, char, next_char_logits]
        predicted_logits = self.model(inputs=input_ids)
        # Only use the last prediction.
        predicted_logits = predicted_logits[:, -1, :]
        predicted_logits = predicted_logits/self.temperature
        # Apply the prediction mask: prevent "[UNK]" from being generated.
        predicted_logits = predicted_logits + self.prediction_mask

        # Sample the output logits to generate token IDs.
        predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
        predicted_ids = tf.squeeze(predicted_ids, axis=-1)

        # Convert from token ids to characters
        predicted_chars = self.chars_from_ids(predicted_ids)

        # Return the characters and model state.
        return predicted_chars

In [217]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [218]:
start = time.time()
# states = None
next_char = tf.constant(['Лопатин:'])
result = [next_char]

for n in range(1000):
    next_char = one_step_model.generate_one_step(next_char)
    
    result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

Лопатин:   нта, про бы плоееня  рюба (н ж? н зв ные 
 най.  м   Дачециму) парари, 
 кла.  адо м, не"н. .. нелька жебелебофедвлаль, повнофихолен пролоя бывет, м   (воде жи      нада  Анекодай пля, Зади пршет.  нак, Ототах!
  Вшиво  (Неноза возвы,  пи    итеть петотышк ду га  завнак В,  вак нюбыйчкледо пя. ко  Вы, ндайчанужащивай Всловы  по Ха  инак. жейчкудит 
  у Тра  ум.  жи ни бы (гови  Ска нахомя  юбизахо т бла...  стетиза   
  мцеля..  Епла.  ида и? демырюбыене   нь Этенетвдиневм;cда..  нышеторшенего Мерала..  тегас тст, Лошланетае.
 тск,  бы па буя, Засто  к. си  А- эт, (стьеетет, пров вуспи,   ужетс поядатст, па, Вайшашето палах    пукас   нюбы, мышлая... сх!
  вдетана умети ча. Ана. Яшивна. со  Ан..) м.)  А чкоме ушихитаспедвняхо -евим ниса т. ни,    в итасе  Яшлоя.  мнья  (Пра Вслу каю, (м Ереня да да! по.  з да,  уде паже бы  мо Любо наененато. (деши тслазажа сли Трашь. праегдалакольhгобидя, гра Я"  вцетво меводаюблы (вне задоядес насе  кукудатам  (в Пофи,  А г...
 Ани   ( ним